In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 1000
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GroupKFold, GroupShuffleSplit, GridSearchCV, StratifiedKFold, train_test_split
from sklearn.model_selection import permutation_test_score
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC
import pickle
import os

## Target: Remitter

In [6]:
df = pd.read_csv("../../data/final_dataset_remitter.csv")
df.head()

,participants_ID,DISC/REP,indication,formal_status,Dataset,Consent,sessSeason,sessTime,Responder,Remitter,...,ever_used_cigarette,hours_since_last_cigarette,ever_used_coffee,hours_since_last_coffee,ever_used_beer,hours_since_last_beer,ever_used_drugs,hours_since_last_drugs,hours_since_last_meal,hours_since_last_sleep
0,sub-87999321,DISCOVERY,MDD,MDD,MDD-rTMS,YES,spring,NaN,1,1,...,0.0,NaN,0.0,NaN,0.0,NaN,1.0,0.0,5.0,7.0
1,sub-88000181,DISCOVERY,MDD,MDD,MDD-rTMS,YES,summer,NaN,0,0,...,0.0,NaN,1.0,9.0,1.0,15.0,1.0,0.0,3.0,7.0
2,sub-88000313,DISCOVERY,MDD,MDD,MDD-rTMS,YES,summer,NaN,1,1,...,1.0,9.0,1.0,9.0,0.0,NaN,1.0,0.0,3.0,9.0
3,sub-88000489,DISCOVERY,MDD,MDD,MDD-rTMS,YES,summer,NaN,1,1,...,0.0,NaN,1.0,5.0,1.0,13.0,1.0,0.0,5.0,9.0
4,sub-88000533,DISCOVERY,MDD,MDD,MDD-rTMS,YES,summer,NaN,1,1,...,1.0,8.0,1.0,9.0,1.0,13.0,1.0,0.0,3.0,5.0


In [7]:
# pull out rows where Dataset = MDD-rTMS
print("All data shape:", df.shape)
df = df[df['Dataset'] == 'MDD-rTMS']
print("Only MDD data shape:", df.shape)

All data shape: (226, 114)
Only MDD data shape: (132, 114)


In [8]:
missing_counts = df.isnull().sum()
missing_counts_sorted = missing_counts.sort_values() # sort
print(missing_counts_sorted)

print("Row missing hearing:", df[df["hearing"].isna()].index)
print("Row missing well:", df[df["well"].isna()].index)

participants_ID                 0
BDI_pre                         0
BDI_post                        0
n_oddb_FN                       0
n_oddb_CN                       0
n_oddb_FP                       0
EO                              0
EC                              0
nrSessions                      0
gender                          0
sessID                          0
Remitter                        0
Responder                       0
n_oddb_CP                       0
sessSeason                      0
Consent                         0
Dataset                         0
formal_status                   0
indication                      0
DISC/REP                        0
age                             0
n_wm_CN                         1
n_wm_CP                         1
n_wm_FP                         1
n_wm_FN                         1
ever_used_coffee                1
well                            1
vision                          1
hearing                         1
rTMS_protocol 

In [9]:
duplicate_ids = df["participants_ID"].value_counts()
duplicate_ids = duplicate_ids[duplicate_ids > 1].index

# print(df[df["participants_ID"].isin(duplicate_ids)])
# noticed that some participants have multiple sessions of data, so we decided
# to only keep the first entry for each participant

df = df.drop_duplicates(subset="participants_ID", keep="first")
print("Shape after keeping only first entry for each participant:", df.shape)

Shape after keeping only first entry for each participant: (124, 114)


### Model & Pipeline

In [12]:
X = df[['age', 'gender', 'education', 'BDI_pre', 'ever_used_drugs']]
y = df['Remitter']

# binary vars float --> str
X = X.copy()
X['gender'] = X['gender'].map({1.0: 'male', 0.0: 'female'})

print(X.shape)
print(X.columns)
print(X.dtypes)
print(X.head())

(124, 5)
Index(['age', 'gender', 'education', 'BDI_pre', 'ever_used_drugs'], dtype='object')
age                float64
gender              object
education          float64
BDI_pre            float64
ever_used_drugs    float64
dtype: object
     age  gender  education  BDI_pre  ever_used_drugs
0  49.66    male       18.0     20.0              1.0
1  45.99  female       11.0     47.0              1.0
2  35.38  female       17.0     21.0              1.0
3  42.36    male       18.0     21.0              1.0
4  45.14  female       17.0     42.0              1.0


In [13]:
# construct preprocesser
# decide which encoder to use on each feature
onehot_ftrs = ['gender', 'ever_used_drugs']
std_ftrs = ['age', 'education', 'BDI_pre']

# one hot pipeline
X['ever_used_drugs'] = X['ever_used_drugs'].fillna(99)
onehot_transformer = Pipeline(steps=[
    ('imputer0', SimpleImputer(strategy='constant')), 
    ('onehot', OneHotEncoder(sparse_output=False, handle_unknown='ignore')),
])

# numeric pipeline with imputation + scaling
numeric_transformer = Pipeline(steps=[
    ('imputer2', SimpleImputer(strategy='median')),  # fills missing values with median
    ('scaler', StandardScaler())                     # scales features
])

# collect all the encoders
preprocessor = ColumnTransformer(
    transformers=[
        ('onehot', onehot_transformer, onehot_ftrs), 
        ('std', numeric_transformer, std_ftrs)])

clf = Pipeline(steps=[('preprocessor', preprocessor)])

In [14]:
# outer split: stratified split test and other
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42) 

print(X_train_val.shape)
print(X_test.shape)

# inner split: stratified k fold with k = 4 (60-20-20)
inner_split = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)

for fold, (train_idx, val_idx) in enumerate(inner_split.split(X_train_val, y_train_val)):
    print(f"Fold {fold+1}")
    X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    print("X_train shape:", X_train.shape)
    print("X_val shape:", X_val.shape)

(99, 5)
(25, 5)
Fold 1
X_train shape: (74, 5)
X_val shape: (25, 5)
Fold 2
X_train shape: (74, 5)
X_val shape: (25, 5)
Fold 3
X_train shape: (74, 5)
X_val shape: (25, 5)
Fold 4
X_train shape: (75, 5)
X_val shape: (24, 5)


In [15]:
# try preprocessor on the last fold
print("Before preprocessing...")
print("X train:", X_train.shape)
print("X val:", X_val.shape)
print("X test:", X_test.shape, "\n")

X_train_prep = clf.fit_transform(X_train)

# relabel the columns after transformation
onehot_feature_names = clf.named_steps['preprocessor'].named_transformers_['onehot'].get_feature_names_out()
new_feature_names = list(onehot_feature_names) + std_ftrs
print(new_feature_names)

X_train_prep = pd.DataFrame(X_train_prep, columns=new_feature_names)
X_val_prep = pd.DataFrame(clf.transform(X_val), columns=new_feature_names)
X_test_prep = pd.DataFrame(clf.transform(X_test), columns=new_feature_names)

# print number of data points after preprocessing
print("After preprocessing...")
print("X train:", X_train_prep.shape)
print("X val:", X_val_prep.shape)
print("X test:", X_test_prep.shape)

Before preprocessing...
X train: (75, 5)
X val: (24, 5)
X test: (25, 5) 

['gender_female', 'gender_male', 'ever_used_drugs_0.0', 'ever_used_drugs_1.0', 'age', 'education', 'BDI_pre']
After preprocessing...
X train: (75, 7)
X val: (24, 7)
X test: (25, 7)


In [16]:
# try simple model on data
log_reg = LogisticRegression(solver='saga', max_iter=100000000)
log_reg.fit(X_train_prep, y_train)
y_pred_val = log_reg.predict(X_val_prep)
y_pred_train = log_reg.predict(X_train_prep)
print("Accuracy on validation set:", accuracy_score(y_val,y_pred_val))
print("Accuracy on training set:", accuracy_score(y_train,y_pred_train))

Accuracy on validation set: 0.5
Accuracy on training set: 0.6133333333333333


In [17]:
# test out permutation_test_score function
log_reg = LogisticRegression(solver='saga', max_iter=100000000)
pipe = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', log_reg)
])

score, perm_scores, p_value = permutation_test_score(
    pipe, 
    X_train_val, 
    y_train_val, 
    scoring="accuracy", 
    cv=inner_split,
    n_permutations=1000,
    random_state=42
)

print("CV accuracy:", score)
print("p-value:", p_value)

CV accuracy: 0.6270833333333333
p-value: 0.017982017982017984
